In [1]:
class Tensor():
    def __init__(self , data):
        self.data = data
        self.grad = 0.0

In [2]:
class Tensor():
    def __init__(self , data , _op='' , label=''):
        self.data = data
        self.grad = 0.0
        self._op = _op
        self.label = label
    
    def __repr__(self):
        return f"Tensor(data={self.data}, grad={self.grad})"

In [3]:
# TESTING
x = Tensor(2.0)
x

Tensor(data=2.0, grad=0.0)

In [4]:
import math

In [6]:
class Tensor():

    def __init__(self , data , _children=() , _op='' , label=''):
        self.data = data
        self.grad = 0.0 
        self._op = _op
        self._backward = lambda: None  # will be overridden
        self._prev = set(_children)
        self.label = label

    def __repr__(self):
        return f"Tensor(data={self.data} , grad={self.grad})"
    
    def __add__(self , other):
        other = other if isinstance(other , Tensor) else Tensor(other)
        rv = Tensor(self.data + other.data , _op='+')

        def _backward():
            self.grad += 1.0 * rv.grad
            other.grad += 1.0 * rv.grad
        rv._backward = _backward
        
        return rv
    
    def __mul__(self, other):
        other = other if isinstance(other , Tensor) else Tensor(other)
        rv = Tensor(self.data * other.data , _op='*')

        def _backward():
            self.grad += other.data * rv.grad
            other.grad += self.data * rv.grad
        rv._backward = _backward

        return rv
    
    def __pow__(self , other):
        assert isinstance(other , (int,float)), "only supporting int/float powers for now"
        rv = Tensor(self.data**other , (self,) , f'**{other}')

        def _backward():
            # dy/dx = n*x^(n-1)*g'(x)
            self.grad += other * (self.data ** (other-1)) * rv.grad
        rv._backward = _backward

        return rv
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self , other):
        return self + (-other)
    
    def __radd__(self , other):  # other + self
        return self + other
    
    def __rmul__(self , other):  # other * self
        return self * other
    
    def __truediv__(self , other):  # self / other
        return self * other**-1
    

    # No dunder methods
    def exp(self):  # e^x
        x = self.data
        rv = Tensor(math.exp(x), (self,) , _op='exp')

        def _backward():
            # dy/dx = e^x
            self.grad += rv.data * rv.data
        rv._backward = _backward

        return rv
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1) / (math.exp(2*x) + 1)
        rv = Tensor(t , (self,) , _op='tanh')

        def _backward():
            # dy/dx = 1-tanh^2(x)*g'(x)
            self.grad += (1 - t**2) * rv.grad
        rv._backward = _backward
        
        return rv
    
    def sine(self):
        x = self.data
        rv = Tensor(x , (self,) , _op='sin')

        def _backward():
            # dy/dx = cos(x)*g'(x)
            self.grad += self.cosine().data * rv.grad
        rv._backward = _backward

        return rv
    
    def cosine(self):
        x = self.data
        rv = Tensor(x , (self,) , _op='cos')

        def _backward():
            # dy/dx = -sin(x)*g'(x)
            self.grad += -self.sine().data * rv.grad
        rv._backward = _backward

        return rv
    
    def sigmoid(self):
        rv = (-(self)).exp()
        rv = Tensor(1 / (1 + rv.data), (self,), 'sigmoid')

        def _backward():
            # dy/dx = sigmoid(1-sigmoid)*g'(x)
            self.grad += (rv.data * (1 - rv.data)) * rv.grad
        rv._backward = _backward

        return rv
    

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # dx/dx
        self.grad = 1.0
        for node in reversed(todo):
            node._backward()

